In [5]:
import gene_symptoms_question_functions as gsf
import importlib
import requests
importlib.reload(gsf)
import pandas as pd

from biothings_explorer.user_query_dispatcher import FindConnection
from biothings_explorer.hint import Hint
ht = Hint()

In [28]:
def get_disease_symptoms(disease_name):
    r = requests.get('http://mydisease.info/v1/query?q=hpo.disease_name:"' + disease_name + '"&fields=hpo')
    res = r.json()
    # print(res)
    result_number = 0
    disease_info = res['hits'][result_number]
    print("disease symptoms for:")
    print(disease_info['hpo']['disease_name'])
    symptoms = []
    hp_ids = []
    hp_symptom_dict = {}
    for x in disease_info['hpo']['phenotype_related_to_disease']:
        # print(x)
#         print(x['aspect'])
#         if((x['aspect'].lower() == 'p')):
        if('frequency' in x):
            r1 = requests.get('https://biothings.ncats.io/hpo/phenotype/' + x['frequency'])
            res1 = r1.json()
#                 print(res1)
#             print(res1['name'])
# #                 in this case, only list symptoms if they are "frequent" or more. 
#             if 'very frequent' in res1['name'].lower():
            r = requests.get('https://biothings.ncats.io/hpo/phenotype/' + x['hpo_id'])
            res = r.json()
#             print(res['name'])
            if(('_id' in res) & ('name' in res)):
                symptoms.append(res['name'].lower())
                hp_ids.append(res['_id'])
                hp_symptom_dict[res['_id']] = {
                    'names' : [res['name'].lower()],
                    'frequency' : res1['name'],
                    'edges_out_count' : 0
                }
            if('synonym' in res):
                for z in res['synonym']:
                    if('EXACT' in z):
                        name = z.split('"')[1].lower()
                        if name not in symptoms: 
                            symptoms.append(name)
                            hp_symptom_dict[res['_id']].append(name)

    print(symptoms)
    return([symptoms,hp_ids,hp_symptom_dict])

In [29]:
current_symtpoms = get_disease_symptoms("Xeroderma Pigmentosum")
xp_symptom_dict = current_symtpoms[2]
symptom_list = current_symtpoms[0]

disease symptoms for:
Xeroderma pigmentosum
['cryptorchidism', 'hypogonadism', 'abnormality of the dentition', 'microcephaly', 'hearing impairment', 'sensorineural hearing impairment', 'strabismus', 'keratitis', 'blepharitis', 'cataract', 'conjunctival telangiectasia', 'photophobia', 'entropion', 'optic atrophy', 'ectropion', 'dry skin', 'hyperkeratosis', 'thin skin', 'cutaneous photosensitivity', 'melanocytic nevus', 'telangiectasia', 'poikiloderma', 'hypermelanotic macule', 'hypopigmented skin patches', 'pterygium', 'thickened skin', 'seizure', 'ataxia', 'spasticity', 'reduced tendon reflexes', 'freckling', 'failure to thrive', 'alopecia', 'fever', 'abnormality of extrapyramidal motor function', 'cerebral cortical atrophy', 'eeg abnormality', 'developmental regression', 'neoplasm', 'delayed skeletal maturation', 'arthralgia', 'melanoma', 'aminoaciduria', 'short stature', 'dermal atrophy', 'craniofacial hyperostosis', 'intellectual disability, progressive', 'opacification of the corne

In [30]:
disease_name = "XERODERMA PIGMENTOSUM"

def get_symtpom_prevalence(hp_symptom_dict):
    for key in hp_symptom_dict:
        edges_out_count = 0
        print("name: " + str(hp_symptom_dict[key]))
#         phen_called = False;
#         dis_called = False;
#         bio_called = False;
        UMLS = '';
        for y in ['PhenotypicFeature','Disease','BiologicalProcess']:
            if y == 'PhenotypicFeature':
                a = ht.query(key)[y]
                if len(a) > 0: 
                    b = a[0]
                    if 'UMLS' in b: 
                        print("YAY UMLS")
                        UMLS = b['UMLS']
                    try: 
                        fc = FindConnection(input_obj=b, output_obj='Gene', intermediate_nodes=None)
                        fc.connect(verbose=False)
                        df = fc.display_table_view()
                        print("phen")
                        print(hp_symptom_dict[key])
                        print(df.shape[0])
                        if(df.shape[0] > 0):
                            df = df[df["output_name"] != disease_name]
                            edges_out_count = edges_out_count + df.shape[0]
#                             print(df.shape)
                        fc = FindConnection(input_obj=b, output_obj='Disease', intermediate_nodes=None)
                        fc.connect(verbose=False)
                        df = fc.display_table_view()
                        print("phen")
                        print(hp_symptom_dict[key])
                        print(df.shape[0])
                        if(df.shape[0] > 0):
                            df = df[df["output_name"] != disease_name]
                            edges_out_count = edges_out_count + df.shape[0]
                    except: 
                        print("Nope")
            if(y =='Disease') | (y == 'BiologicalProcess'):
                for z in hp_symptom_dict[key]["names"]:
                    if((y == 'Disease') & (len(UMLS) > 0)): 
                        a = ht.query(UMLS)[y]
                    else:
                        a = ht.query(z)[y]
                    print(a)
                    for b in a: 
                        if b['name'].lower() == z:
                            print('match')
                            print(b)
                            print(z)
                            try: 
                                fc = FindConnection(input_obj=b, output_obj='Gene', intermediate_nodes=None)
                                fc.connect(verbose=False)
                                df = fc.display_table_view()
                                print("BD")
                                print(df.shape[0])
                                if(df.shape[0] > 0):
                                    df = df[df["output_name"] != disease_name]
                                    edges_out_count = edges_out_count + df.shape[0]
                                fc = FindConnection(input_obj=b, output_obj='Disease', intermediate_nodes=None)
                                fc.connect(verbose=False)
                                df = fc.display_table_view()
                                print("BD")
                                print(df.shape[0])
                                if(df.shape[0] > 0):
                                    df = df[df["output_name"] != disease_name]
                                    edges_out_count = edges_out_count + df.shape[0]
    #                             print(df.shape)
                            except: 
                                print("Nope")

        hp_symptom_dict[key]["edges_out_count"] = edges_out_count
    return(hp_symptom_dict)
            
xp_symptom_dict = get_symtpom_prevalence(xp_symptom_dict) 



name: {'names': ['cryptorchidism'], 'frequency': 'Frequent', 'edges_out_count': 0}
YAY UMLS
API 2.1 semmed_phenotype failed
API 2.2 semmed_phenotype failed
API 2.5 semmed_phenotype failed
API 2.6 semmed_phenotype failed
API 2.8 semmed_phenotype failed
API 2.4 semmed_phenotype failed
API 2.3 semmed_phenotype failed
API 2.7 semmed_phenotype failed
API 2.9 semmed_phenotype failed
API 2.12 semmed_phenotype failed
API 2.13 semmed_phenotype failed
API 2.10 semmed_phenotype failed
API 2.11 semmed_phenotype failed
phen
{'names': ['cryptorchidism'], 'frequency': 'Frequent', 'edges_out_count': 0}
331
API 1.2 semmed_phenotype failed
API 1.3 semmed_phenotype failed
API 1.1 semmed_phenotype failed
API 1.7 semmed_phenotype failed
API 1.9 semmed_phenotype failed
API 1.6 semmed_phenotype failed
API 1.4 semmed_phenotype failed
API 1.8 semmed_phenotype failed
API 1.5 semmed_phenotype failed
API 1.13 semmed_phenotype failed
API 1.14 semmed_phenotype failed
API 1.10 semmed_phenotype failed
API 1.11 semmed

API 1.1 semmed_phenotype failed
API 1.2 semmed_phenotype failed
API 1.7 semmed_phenotype failed
API 1.6 semmed_phenotype failed
API 1.11 semmed_phenotype failed
API 1.5 semmed_phenotype failed
API 1.4 semmed_phenotype failed
API 1.3 semmed_phenotype failed
API 1.13 semmed_phenotype failed
API 1.10 semmed_phenotype failed
API 1.8 semmed_phenotype failed
API 1.12 semmed_phenotype failed
API 1.9 semmed_phenotype failed
API 1.16 semmed_phenotype failed
API 1.14 semmed_phenotype failed
API 1.19 semmed_phenotype failed
API 1.15 semmed_phenotype failed
API 1.18 semmed_phenotype failed
API 1.17 semmed_phenotype failed
phen
{'names': ['sensorineural hearing impairment'], 'frequency': 'Frequent', 'edges_out_count': 0}
343
[{'MONDO': 'MONDO:0019497', 'DOID': 'DOID:0050563', 'UMLS': 'CN043648', 'name': 'nonsyndromic genetic deafness', 'MESH': 'C580334', 'ORPHANET': '87884', 'primary': {'identifier': 'MONDO', 'cls': 'Disease', 'value': 'MONDO:0019497'}, 'display': 'MONDO(MONDO:0019497) DOID(DOID:00

phen
{'names': ['cataract'], 'frequency': 'Frequent', 'edges_out_count': 0}
346
[{'MONDO': 'MONDO:0020224', 'UMLS': 'C0086543', 'name': 'obsolete rare cataract', 'ORPHANET': '98640', 'primary': {'identifier': 'MONDO', 'cls': 'Disease', 'value': 'MONDO:0020224'}, 'display': 'MONDO(MONDO:0020224) ORPHANET(98640) UMLS(C0086543) name(obsolete rare cataract)', 'type': 'Disease'}]
[]
name: {'names': ['conjunctival telangiectasia'], 'frequency': 'Very frequent', 'edges_out_count': 0}
YAY UMLS
API 2.1 semmed_phenotype failed
API 2.7 semmed_phenotype failed
API 2.2 semmed_phenotype failed
API 2.3 semmed_phenotype failed
API 2.10 semmed_phenotype failed
API 2.9 semmed_phenotype failed
API 2.4 semmed_phenotype failed
API 2.5 semmed_phenotype failed
API 2.11 semmed_phenotype failed
API 2.6 semmed_phenotype failed
API 2.8 semmed_phenotype failed
API 2.12 semmed_phenotype failed
API 2.13 semmed_phenotype failed
phen
{'names': ['conjunctival telangiectasia'], 'frequency': 'Very frequent', 'edges_out_

[]
name: {'names': ['dry skin'], 'frequency': 'Very frequent', 'edges_out_count': 0}
YAY UMLS
API 2.2 semmed_phenotype failed
API 2.6 semmed_phenotype failed
API 2.3 semmed_phenotype failed
API 2.1 semmed_phenotype failed
API 2.4 semmed_phenotype failed
API 2.8 semmed_phenotype failed
API 2.7 semmed_phenotype failed
API 2.5 semmed_phenotype failed
API 2.13 semmed_phenotype failed
API 2.10 semmed_phenotype failed
API 2.11 semmed_phenotype failed
API 2.9 semmed_phenotype failed
API 2.12 semmed_phenotype failed
phen
{'names': ['dry skin'], 'frequency': 'Very frequent', 'edges_out_count': 0}
147
API 1.2 semmed_phenotype failed
API 1.7 semmed_phenotype failed
API 1.6 semmed_phenotype failed
API 1.4 semmed_phenotype failed
API 1.1 semmed_phenotype failed
API 1.5 semmed_phenotype failed
API 1.9 semmed_phenotype failed
API 1.3 semmed_phenotype failed
API 1.11 semmed_phenotype failed
API 1.12 semmed_phenotype failed
API 1.10 semmed_phenotype failed
API 1.8 semmed_phenotype failed
API 1.15 semme

phen
{'names': ['telangiectasia'], 'frequency': 'Very frequent', 'edges_out_count': 0}
165
[{'MONDO': 'MONDO:0001576', 'DOID': 'DOID:1272', 'UMLS': 'C0039446', 'name': 'telangiectasis', 'MESH': 'D013684', 'primary': {'identifier': 'MONDO', 'cls': 'Disease', 'value': 'MONDO:0001576'}, 'display': 'MONDO(MONDO:0001576) DOID(DOID:1272) UMLS(C0039446) MESH(D013684) name(telangiectasis)', 'type': 'Disease'}]
[]
name: {'names': ['poikiloderma'], 'frequency': 'Very frequent', 'edges_out_count': 0}
YAY UMLS
API 2.6 semmed_phenotype failed
API 2.1 semmed_phenotype failed
API 2.2 semmed_phenotype failed
API 2.4 semmed_phenotype failed
API 2.3 semmed_phenotype failed
API 2.7 semmed_phenotype failed
API 2.5 semmed_phenotype failed
API 2.10 semmed_phenotype failed
API 2.11 semmed_phenotype failed
API 2.8 semmed_phenotype failed
API 2.9 semmed_phenotype failed
API 2.12 semmed_phenotype failed
API 2.13 semmed_phenotype failed
phen
{'names': ['poikiloderma'], 'frequency': 'Very frequent', 'edges_out_co

...

phen
{'names': ['cerebral cortical atrophy'], 'frequency': 'Occasional', 'edges_out_count': 0}
689
[{'UMLS': 'C0235946', 'name': 'Cerebral atrophy', 'primary': {'identifier': 'UMLS', 'cls': 'Disease', 'value': 'C0235946'}, 'display': 'UMLS(C0235946) name(Cerebral atrophy)', 'type': 'Disease'}]
[]
name: {'names': ['eeg abnormality'], 'frequency': 'Very frequent', 'edges_out_count': 0}
YAY UMLS
API 2.3 semmed_phenotype failed
API 2.5 semmed_phenotype failed
API 2.2 semmed_phenotype failed
API 2.4 semmed_phenotype failed
API 2.1 semmed_phenotype failed
API 2.8 semmed_phenotype failed
API 2.10 semmed_phenotype failed
API 2.6 semmed_phenotype failed
API 2.7 semmed_phenotype failed
API 2.13 semmed_phenotype failed
API 2.12 semmed_phenotype failed
API 2.9 semmed_phenotype failed
API 2.11 semmed_phenotype failed
phen
{'names': ['eeg abnormality'], 'frequency': 'Very frequent', 'edges_out_count': 0}
182
API 1.1 semmed_phenotype failed
API 1.3 semmed_phenotype failed
API 1.4 semmed_phenotype fai

BD
37
BD
469
[]
name: {'names': ['melanoma'], 'frequency': 'Frequent', 'edges_out_count': 0}
YAY UMLS
API 2.1 semmed_phenotype failed
API 2.3 semmed_phenotype failed
API 2.2 semmed_phenotype failed
API 2.5 semmed_phenotype failed
API 2.6 semmed_phenotype failed
API 2.7 semmed_phenotype failed
API 2.8 semmed_phenotype failed
API 2.9 semmed_phenotype failed
API 2.4 semmed_phenotype failed
API 2.10 semmed_phenotype failed
API 2.12 semmed_phenotype failed
API 2.13 semmed_phenotype failed
API 2.11 semmed_phenotype failed
phen
{'names': ['melanoma'], 'frequency': 'Frequent', 'edges_out_count': 0}
59
API 1.1 semmed_phenotype failed
API 1.2 semmed_phenotype failed
API 1.5 semmed_phenotype failed
API 1.10 semmed_phenotype failed
API 1.4 semmed_phenotype failed
API 1.8 semmed_phenotype failed
API 1.9 semmed_phenotype failed
API 1.11 semmed_phenotype failed
API 1.7 semmed_phenotype failed
API 1.3 semmed_phenotype failed
API 1.6 semmed_phenotype failed
API 1.14 semmed_phenotype failed
API 1.16 sem

phen
{'names': ['craniofacial hyperostosis'], 'frequency': 'Occasional', 'edges_out_count': 0}
32
[]
[]
name: {'names': ['intellectual disability, progressive'], 'frequency': 'Very frequent', 'edges_out_count': 0}
YAY UMLS
API 2.3 semmed_phenotype failed
API 2.4 semmed_phenotype failed
API 2.1 semmed_phenotype failed
API 2.2 semmed_phenotype failed
API 2.6 semmed_phenotype failed
API 2.7 semmed_phenotype failed
API 2.5 semmed_phenotype failed
API 2.8 semmed_phenotype failed
API 2.9 semmed_phenotype failed
API 2.12 semmed_phenotype failed
API 2.11 semmed_phenotype failed
API 2.13 semmed_phenotype failed
API 2.10 semmed_phenotype failed
phen
{'names': ['intellectual disability, progressive'], 'frequency': 'Very frequent', 'edges_out_count': 0}
103
API 1.4 semmed_phenotype failed
API 1.5 semmed_phenotype failed
API 1.3 semmed_phenotype failed
API 1.2 semmed_phenotype failed
API 1.7 semmed_phenotype failed
API 1.9 semmed_phenotype failed
API 1.1 semmed_phenotype failed
API 1.6 semmed_pheno

phen
{'names': ['flat nasal alae'], 'frequency': 'Occasional', 'edges_out_count': 0}
3
[]
[]
name: {'names': ['erythema'], 'frequency': 'Frequent', 'edges_out_count': 0}
YAY UMLS
API 2.4 semmed_phenotype failed
API 2.2 semmed_phenotype failed
API 2.1 semmed_phenotype failed
API 2.5 semmed_phenotype failed
API 2.3 semmed_phenotype failed
API 2.6 semmed_phenotype failed
API 2.8 semmed_phenotype failed
API 2.7 semmed_phenotype failed
API 2.10 semmed_phenotype failed
API 2.11 semmed_phenotype failed
API 2.13 semmed_phenotype failed
API 2.9 semmed_phenotype failed
API 2.12 semmed_phenotype failed
phen
{'names': ['erythema'], 'frequency': 'Frequent', 'edges_out_count': 0}
105
API 1.1 semmed_phenotype failed
API 1.2 semmed_phenotype failed
API 1.4 semmed_phenotype failed
API 1.7 semmed_phenotype failed
API 1.3 semmed_phenotype failed
API 1.12 semmed_phenotype failed
API 1.14 semmed_phenotype failed
API 1.5 semmed_phenotype failed
API 1.13 semmed_phenotype failed
API 1.6 semmed_phenotype faile

phen
{'names': ['neoplasm of the eye'], 'frequency': 'Occasional', 'edges_out_count': 0}
42
API 1.5 semmed_phenotype failed
API 1.3 semmed_phenotype failed
API 1.9 semmed_phenotype failed
API 1.10 semmed_phenotype failed
API 1.2 semmed_phenotype failed
API 1.7 semmed_phenotype failed
API 1.8 semmed_phenotype failed
API 1.11 semmed_phenotype failed
API 1.4 semmed_phenotype failed
API 1.12 semmed_phenotype failed
API 1.6 semmed_phenotype failed
API 1.15 semmed_phenotype failed
API 1.13 semmed_phenotype failed
API 1.14 semmed_phenotype failed
API 1.19 semmed_phenotype failed
API 1.16 semmed_phenotype failed
API 1.17 semmed_phenotype failed
API 1.18 semmed_phenotype failed
API 1.1 semmed_phenotype failed
phen
{'names': ['neoplasm of the eye'], 'frequency': 'Occasional', 'edges_out_count': 0}
32
[]
[]
name: {'names': ['cognitive impairment'], 'frequency': 'Very frequent', 'edges_out_count': 0}
YAY UMLS
API 2.3 semmed_phenotype failed
API 2.6 semmed_phenotype failed
API 2.1 semmed_phenotype 

In [31]:
xp_symptom_dict

{'HP:0000028': {'names': ['cryptorchidism'],
  'frequency': 'Frequent',
  'edges_out_count': 676},
 'HP:0000135': {'names': ['hypogonadism'],
  'frequency': 'Very frequent',
  'edges_out_count': 1264},
 'HP:0000164': {'names': ['abnormality of the dentition'],
  'frequency': 'Very frequent',
  'edges_out_count': 367},
 'HP:0000252': {'names': ['microcephaly'],
  'frequency': 'Occasional',
  'edges_out_count': 712},
 'HP:0000365': {'names': ['hearing impairment'],
  'frequency': 'Occasional',
  'edges_out_count': 576},
 'HP:0000407': {'names': ['sensorineural hearing impairment'],
  'frequency': 'Frequent',
  'edges_out_count': 650},
 'HP:0000486': {'names': ['strabismus'],
  'frequency': 'Frequent',
  'edges_out_count': 693},
 'HP:0000491': {'names': ['keratitis'],
  'frequency': 'Frequent',
  'edges_out_count': 416},
 'HP:0000498': {'names': ['blepharitis'],
  'frequency': 'Occasional',
  'edges_out_count': 131},
 'HP:0000518': {'names': ['cataract'],
  'frequency': 'Frequent',
  'edg

In [32]:
# for key in xp_symptom_dict:
#     data = {'row_1': [3, 2, 1, 0], 'row_2': ['a', 'b', 'c', 'd']}
symptoms_df = pd.DataFrame.from_dict(xp_symptom_dict, orient='index').sort_values(by=['edges_out_count'])
symptoms_df = pd.concat([symptoms_df[symptoms_df["frequency"] == "Very frequent"],
               symptoms_df[symptoms_df["frequency"] == "Frequent"],
               symptoms_df[symptoms_df["frequency"] == "Occasional"],
               symptoms_df[symptoms_df["frequency"] == "Rare"],
               symptoms_df[symptoms_df["frequency"] == "Unknown"]
              ])
symptoms_df

,names,frequency,edges_out_count
HP:0000524,[conjunctival telangiectasia],Very frequent,52
HP:0001029,[poikiloderma],Very frequent,59
HP:0001480,[freckling],Very frequent,92
HP:0100585,[telangiectasia of the skin],Very frequent,136
HP:0000963,[thin skin],Very frequent,219
HP:0000992,[cutaneous photosensitivity],Very frequent,220
HP:0006887,"[intellectual disability, progressive]",Very frequent,236
HP:0001072,[thickened skin],Very frequent,242
HP:0001009,[telangiectasia],Very frequent,289
HP:0100543,[cognitive impairment],Very frequent,315


In [33]:
# symptoms_df.to_csv("xp_symptoms_df.csv")

In [11]:
# symptoms_df = pd.read_csv("xp_symptoms_df.csv",index_col=0) 
symptoms_df

,names,frequency,edges_out_count
HP:0000524,['conjunctival telangiectasia'],Very frequent,52
HP:0001029,['poikiloderma'],Very frequent,59
HP:0001480,['freckling'],Very frequent,92
HP:0100585,['telangiectasia of the skin'],Very frequent,136
HP:0000963,['thin skin'],Very frequent,219
HP:0000992,['cutaneous photosensitivity'],Very frequent,220
HP:0006887,"['intellectual disability, progressive']",Very frequent,236
HP:0001072,['thickened skin'],Very frequent,242
HP:0001009,['telangiectasia'],Very frequent,289
HP:0100543,['cognitive impairment'],Very frequent,315


In [19]:
individual_symptom_scores = [];
import math

for index,x in symptoms_df.iterrows():
    if(x["frequency"] == "Very frequent"):
        score = 20 / math.sqrt(int(x["edges_out_count"]))
    elif(x["frequency"] == "Frequent"):
        score = 15 / math.sqrt(int(x["edges_out_count"]))
    elif(x["frequency"] == "Occasional"):
        score = 10 / math.sqrt(int(x["edges_out_count"]))
    elif(x["frequency"] == "Rare"):
        score = 5 / math.sqrt(int(x["edges_out_count"]))
    elif(x["frequency"] == "Unknown"):
        score = 5 / math.sqrt(int(x["edges_out_count"]))
    individual_symptom_scores.append(score)

print(len(individual_symptom_scores))
symptoms_df["ISS"] = individual_symptom_scores

59


In [20]:
symptoms_df

,names,frequency,edges_out_count,ISS
HP:0000524,['conjunctival telangiectasia'],Very frequent,52,2.773501
HP:0001029,['poikiloderma'],Very frequent,59,2.603778
HP:0001480,['freckling'],Very frequent,92,2.085144
HP:0100585,['telangiectasia of the skin'],Very frequent,136,1.714986
HP:0000963,['thin skin'],Very frequent,219,1.351475
HP:0000992,['cutaneous photosensitivity'],Very frequent,220,1.348400
HP:0006887,"['intellectual disability, progressive']",Very frequent,236,1.301889
HP:0001072,['thickened skin'],Very frequent,242,1.285649
HP:0001009,['telangiectasia'],Very frequent,289,1.176471
HP:0100543,['cognitive impairment'],Very frequent,315,1.126872


In [21]:
symptoms_df.to_csv("xp_symptoms_df.csv")